# Traffic Orchestrator Decision Support for airport users 

Use the SBC approach to model queues within the airport that are aligned with flow rates from the macrosimulation

Author Ilias Parmaksizoglou


In [10]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use(['science',"no-latex"])
plt.rc('font',**{'family':'sans-serif','sans-serif':['Century Gothic']})
plt.rcParams['figure.figsize'] = [25, 12]


In [11]:
def get_approx_measures(l,mu,c):

    '''
    Generates approximate measures to be used in the SBC approach:

    Args:
        l (np.array) : arrival rates per unit of time.
        mu (np.array) : processing rate of server per unit of time.
        c (np.array) : available servers per unit of time.

    Returns:
        P (float): Probability of the system having a backlog (not serving everyone at this timestep)
        E (float): Utilization of the servers. (probability of the system being empty)
        l_mar (float) : Modified arrival rate for the SBC approach
    '''

    P = (l/mu)**c/(math.factorial(c)*(sum([((l/mu)**x)/(math.factorial(x)) for x in range(0,c+1)])))
    E = (l-P*l)/(c*mu)
    l_mar = c*mu*E
    return P,E,l_mar

In [12]:
def get_mmc_measures(l_mar,mu,c):

    '''
    Generates approximate measures from the M/M/C queue using the formulas from
    Queueing Networks And Markov Chains - Modelling And Performance Evaluation With Computer Science Applications (p.218,Eq.6.27-28):

    Args:
        l_mar (np.array) : modified arrival rates per unit of time.
        mu (np.array) : processing rate of server per unit of time.
        c (np.array) : available servers per unit of time.

    Returns:
        L_q (float): Number of people in the queue
        W (float): Waiting time in the queue in minutes
    '''

    rho = l_mar/(c*mu)
    cons = ((c*rho)**c)/(math.factorial(c)*((1-rho)))
    P_0 = sum([((c*rho)**x)/(math.factorial(x))  for x in range(0,c)]) + cons
    P_0 = (P_0)**(-1)
    Lq = (P_0 *(l_mar/mu)**c*rho)/(math.factorial(c)*((1-rho)**2))
    W = Lq/l_mar
    return Lq,W

In [13]:
def get_mdc_measures(l_mar,mu,c,W_m):

    '''
    Generates approximate measures from the M/D/C queue using the formulas from
    Queueing Networks And Markov Chains - Modelling And Performance Evaluation With Computer Science Applications (p.231,Eq.6.82):

    Args:
        l_mar (np.array) : modified arrival rates per unit of time.
        mu (np.array) : processing rate of server per unit of time.
        c (np.array) : available servers per unit of time.

    Returns:
        L_q (float): Number of people in the queue
        W (float): Waiting time in the queue in minutes

    '''

    rho = l_mar/(c*mu)
    nc = (1 + (1-rho)*(c-1)*((4+5*c)**0.5-2)/(16*rho*c))**(-1)
    W = 0.5*(1/(nc))*W_m
    Lq = W*l_mar
    return Lq,W

In [14]:

def get_arrival_rates(PT_DF,CARS_DF,queue_chars,point):

    '''
    Isolates arrival rates at specific point in the chain

    Args:
        PT_DF (pd.DataFrame) : Arrivals flows from PT; stored as xlsx file
        PT_CARS (pd.DataFrame) : Arrivals flows from Cars & Taxi; stored as xlsx file
        queue_chars (dict) : Queue characteristics for specific points in the airport
        point (str) : point in the airport (SELF-CHECK, BAGS, PASSPORT).

    Returns:
        t (list): Timesteps in the examined queue
        l (np.array): Flow rates in the examined queue per timestep
        mu (np.array): Processing rate in the examined queue per timestep
        c (np.array): Available servers in the examined queue per timestep

    '''

    # Concat DFs and sort by activation time               
    df = pd.concat([CARS_DF,PT_DF])
    df = df.sort_values(by = ["END_TIME"])
    df['END_TIME'] = df['END_TIME'] .dt.hour*60 + df['END_TIME'] .dt.minute

    # Isolate traffic at examined point
    if point == "BAGS":
        df_mod = df[df['SELF-CHECK'] == 0]
        df_mod = df_mod[df_mod[point] == 1]
    elif point == "X-RAY":
        df_mod = df[df['BAGS'] == 0] 
        df_mod = df_mod[df_mod['SELF-CHECK'] == 0]
    else:
        df_mod = df[df[point] == 1]

    df_mod = df_mod.groupby('END_TIME')['NO_PERSONS'].sum()
    df_mod = df_mod.reindex(list(range(df_mod.index.min(),df_mod.index.max()+1)),fill_value=1e-16)

    #Initialise timesteps and lambda
    t = df_mod.index.tolist()
    l = df_mod.tolist()
    
    # Concentrate arrivals to timesteps related to mean service time
    t = [x for x in t if x % queue_chars[point]['mean_service_time'] == t[0] % queue_chars[point]['mean_service_time']]
    l = np.array([sum(l[x:x+queue_chars[point]['mean_service_time']]) for x in range(len(l)) if x % queue_chars[point]['mean_service_time'] == 0])
    mu = np.ones(l.shape)/queue_chars[point]['mean_service_time']
    c = (np.ones(l.shape)*queue_chars[point]['servers']).astype(int)

    return t,l,mu,c



In [15]:

def plot_queue_stats(t,E,Lq,Wq,point):
    # Initialise the subplot function using number of rows and columns
    figure, axis = plt.subplots(3, 1)
    
    # For Utilization
    axis[0].plot(t, E)
    axis[0].set_title(f"Utilization of servers per minute in {point}")
    axis[0].set(xlabel="", ylabel="Utilization")
    
    # For Queue Lengths
    axis[1].plot(t, Lq)
    axis[1].set_title(f"Length of Queue(s) per minute in {point}")
    axis[1].set(xlabel="", ylabel="Persons")
    
    # For Waiting time is Queue
    axis[2].plot(t, Wq)
    axis[2].set_title(f"Waiting time in Queue(s) per minute in {point}")
    axis[2].set(xlabel="Minutes", ylabel="Seconds")


    # Combine all the operations and display
    plt.show()

In [16]:
def get_queues(PT_DF,CARS_DF,queue_chars):
    next_point = {"SELF-CHECK":{},"BAGS":{},"X-RAY":{}}
    for point in queue_chars.keys():

        # Get Exlusive traffic (starting point is point)
        t,l,mu,c = get_arrival_rates(PT_DF,CARS_DF,queue_chars,point=point)
        s2,mean_service = queue_chars[point]['S_SQUARED'], queue_chars[point]['mean_service_time']

        # Initialize probabilities of backlog
        P=np.zeros(l.shape)
        l_t= np.zeros(l.shape)
        E = np.zeros(l.shape)
        l_mar = np.zeros(l.shape)

        
        # Add Incoming traffic from other points
        index = 0
        for step in next_point[point].keys():
            if step in t:
                index = t.index(step)
                l[index]+=next_point[point][step]
            else:
                l[index]+=next_point[point][step]
        
        # Get state at time-window 0
        l_t[0] = l[0]
        P[0],E[0],l_mar[0]= get_approx_measures(l_t[0],mu[0],c[0])

        # Store M/m/c measures
        Lq_M,Wq_M = [],[]

        # Store M/d/c measures
        Wq_D,Lq_D = [],[]

        # Store M/g/c (Final) measures
        Lq,Wq,Ex_q = [],[],[]

        for idx in range(l.shape[0]):
            if idx != 0:       
                l_t[idx] = l[idx] + P[idx-1]*l_t[idx-1] # Modify l tilde
                P[idx],E[idx],l_mar[idx]= get_approx_measures(l_t[idx],mu[idx],c[idx])

            # Store M/M/C
            Lq_m,Wq_m = get_mmc_measures(l_mar[idx],mu[idx],c[idx])
            Lq_M.append(Lq_m)
            Wq_M.append(Wq_m)

            # Store M/D/c
            Lq_d,Wq_d = get_mdc_measures(l_mar[idx],mu[idx],c[idx],Wq_m) 
            Wq_D.append(Wq_d)
            Lq_D.append(Lq_d)

            # Cosmetatos Approximation p.231 (6.81)
            W = (s2*Wq_m + (1-s2)*Wq_d)
            L = W*l_mar[idx]
            Wq.append(60*W)
            Lq.append(L)
            
            for idx2,x in enumerate(queue_chars[point]['trans_point']):
                if round(t[idx]+mean_service+W) not in next_point[x].keys():
                    next_point[x][round(t[idx]+mean_service+W)] = round(queue_chars[point]['trans'][idx2]*l[idx])
                else:
                    next_point[x][round(t[idx]+mean_service+W)] += round(queue_chars[point]['trans'][idx2]*l[idx])

        if point == "X-RAY":
            plot_queue_stats(t,E,Lq,Wq,point)

In [19]:
# Input arrivals from file
#CARS_DF = pd.read_excel('data/passengers/June/1-6-2023/CAR_270_330.xlsx')
PT_DF = pd.read_excel('data/passengers/June/1-6-2023/agents_300_390.xlsx')
CARS_DF = []

# Set Processing rates
queue_chars = {"BAGS"      : {'mean_service_time':3, 'servers' : 40, "S_SQUARED": (1-5)**2/12, 'trans' : [1],"trans_point" : ["X-RAY"]},
               "X-RAY"     : {'mean_service_time':1, 'servers' : 10, "S_SQUARED": (0.5-1.5)**2/12,'trans' : [],"trans_point" : []}}

get_queues(PT_DF,CARS_DF,queue_chars)


FileNotFoundError: [Errno 2] No such file or directory: 'data/passengers/June/1-6-2023/agents_300_390.xlsx'